In [ ]:
%matplotlib inline

In [ ]:
!pip install dgl

In [ ]:
import dgl

def build_karate_club_graph():
    g = dgl.DGLGraph()
    # add 34 nodes into the graph; nodes are labeled from 0~33
    g.add_nodes(34)
    # all 78 edges as a list of tuples
    edge_list = [(1, 0), (2, 0), (2, 1), (3, 0), (3, 1), (3, 2),
        (4, 0), (5, 0), (6, 0), (6, 4), (6, 5), (7, 0), (7, 1),
        (7, 2), (7, 3), (8, 0), (8, 2), (9, 2), (10, 0), (10, 4),
        (10, 5), (11, 0), (12, 0), (12, 3), (13, 0), (13, 1), (13, 2),
        (13, 3), (16, 5), (16, 6), (17, 0), (17, 1), (19, 0), (19, 1),
        (21, 0), (21, 1), (25, 23), (25, 24), (27, 2), (27, 23),
        (27, 24), (28, 2), (29, 23), (29, 26), (30, 1), (30, 8),
        (31, 0), (31, 24), (31, 25), (31, 28), (32, 2), (32, 8),
        (32, 14), (32, 15), (32, 18), (32, 20), (32, 22), (32, 23),
        (32, 29), (32, 30), (32, 31), (33, 8), (33, 9), (33, 13),
        (33, 14), (33, 15), (33, 18), (33, 19), (33, 20), (33, 22),
        (33, 23), (33, 26), (33, 27), (33, 28), (33, 29), (33, 30),
        (33, 31), (33, 32)]
    # add edges two lists of nodes: src and dst
    src, dst = tuple(zip(*edge_list))
    g.add_edges(src, dst)
    # edges are directional in DGL; make them bi-directional
    g.add_edges(dst, src)

    return g

We can print out the number of nodes and edges in our newly constructed graph:



In [ ]:
G = build_karate_club_graph()
print('We have %d nodes.' % G.number_of_nodes())
print('We have %d edges.' % G.number_of_edges())
print(G)

We can also visualize the graph by converting it to a `networkx
<https://networkx.github.io/documentation/stable/>`_ graph:



In [ ]:
import networkx as nx
# Since the actual graph is undirected, we convert it for visualization
# purpose.
nx_G = G.to_networkx().to_undirected()
# Kamada-Kawaii layout usually looks pretty for arbitrary graphs
pos = nx.kamada_kawai_layout(nx_G)
print(pos)
print("mm",nx_G)
nx.draw(nx_G, pos, with_labels=True, node_color=[[.7, .7, .7]])

Step 2: assign features to nodes or edges
--------------------------------------------
Graph neural networks associate features with nodes and edges for training.
For our classification example, we assign each node's an input feature as a one-hot vector:
node $v_i$'s feature vector is $[0,\ldots,1,\dots,0]$,
where the $i^{th}$ position is one.

In DGL, we can add features for all nodes at once, using a feature tensor that
batches node features along the first dimension. This code below adds the one-hot
feature for all nodes:



In [ ]:
import torch

G.ndata['feat'] = torch.eye(34)

We can print out the node features to verify:



In [ ]:
# print out node 2's input feature
print(G.nodes[2].data['feat'])

# print out node 10 and 11's input features
print(G.nodes[[10, 11]].data['feat'])

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
# Define the message & reduce function
# NOTE: we ignore the GCN's normalization constant c_ij for this tutorial.
def gcn_message(edges):
    # The argument is a batch of edges.
    # This computes a (batch of) message called 'msg' using the source node's feature 'h'.
    return {'msg' : edges.src['h']}

def gcn_reduce(nodes):
    # The argument is a batch of nodes.
    # This computes the new 'h' features by summing received 'msg' in each node's mailbox.
    return {'h' : torch.sum(nodes.mailbox['msg'], dim=1)}

# Define the GCNLayer module
class GCNLayer(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)

    def forward(self, g, inputs):
        # g is the graph and the inputs is the input node features
        # first set the node features
        g.ndata['h'] = inputs
        # trigger message passing on all edges 
        g.send(g.edges(), gcn_message)
        # trigger aggregation at all nodes
        g.recv(g.nodes(), gcn_reduce)
        # get the result node features
        h = g.ndata.pop('h')
        # perform linear transformation
        return self.linear(h)

In general, the nodes send information computed via the *message functions*,
and aggregates incoming information with the *reduce functions*.

We then define a deeper GCN model that contains two GCN layers:



In [ ]:
# Define a 2-layer GCN model
class GCN(nn.Module):
    def __init__(self, in_feats, hidden_size, num_classes):
        super(GCN, self).__init__()
        self.gcn1 = GCNLayer(in_feats, hidden_size)
        self.gcn2 = GCNLayer(hidden_size, num_classes)

    def forward(self, g, inputs):
        #h = torch.tanh(inputs)
        h = self.gcn1(g, inputs)
        h = torch.tanh(h)
        h = self.gcn2(g, h)
        h = torch.tanh(h)
        return h
# The first layer transforms input features of size of 34 to a hidden size of 5.
# The second layer transforms the hidden layer and producres output features of
# size 2, corresponding to the two groups of the karate club.
net = GCN(34, 4, 4)
print(net)

Step 4: data preparation and initialization
-------------------------------------------

We use one-hot vectors to initialize the node features. Since this is a
semi-supervised setting, only the instructor (node 0) and the club president
(node 33) are assigned labels. The implementation is available as follow.



In [ ]:
inputs = torch.eye(34)
print(inputs)

labeled_nodes = torch.tensor([6,12, 22,33])  # only the instructor and the president nodes are labeled
print(labeled_nodes)
labels = torch.tensor([0, 1,2,3])
print(labels)# their labels are different

Step 5: train then visualize
----------------------------
The training loop is exactly the same as other PyTorch models.
We (1) create an optimizer, (2) feed the inputs to the model,
(3) calculate the loss and (4) use autograd to optimize the model.



In [ ]:
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
all_logits = []
for epoch in range(201):
    logits = net(G, inputs)
    # we save the logits for visualization later
    #print("log",logits)
    all_logits.append(logits.detach())
  
    logp = F.log_softmax(logits, 1)
    # we only compute loss for labeled nodes
    loss = F.nll_loss(logp[labeled_nodes], labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print('Epoch %d | Loss: %.4f' % (epoch, loss.item()))
#print(all_logits[0][0])

This is a rather toy example, so it does not even have a validation or test
set. Instead, Since the model produces an output feature of size 2 for each node, we can
visualize by plotting the output feature in a 2D space.
The following code animates the training process from initial guess
(where the nodes are not classified correctly at all) to the end
(where the nodes are linearly separable).



In [ ]:
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
tsne_data1=[]
i=0
epochess=[]
while(i<201):
    outp=all_logits[i]
    data_1000=outp
#labels_1000=labels
#print(labels)
    md=TSNE(n_components=2,random_state=0,perplexity = 2,learning_rate  = 100, n_iter=5000)

    tsne_data=md.fit_transform(data_1000)
    tsne_data1.append(tsne_data.tolist())
    epochess.append(i)
    i=i+1
#print(len(tsne_data1))

In [ ]:
labels=[0, 1, 1, 1, 0, 0, 0, 1, 1, 2, 0, 1, 1, 1, 2, 2, 0, 1, 2, 1, 2, 1, 2, 2, 3, 3, 2, 2, 2, 2, 2, 2, 3, 3]

In [ ]:
print(len(all_logs[0][0]))

In [ ]:

import matplotlib.animation as animation
import matplotlib.pyplot as plt
pos = nx.kamada_kawai_layout(nx_G)
epoc=[]
group=[]
def draw(i):
    cls1color = '#9B59B6'
    cls2color = '#F7DC6F'
    cls3color = '#A2D9CE'
    cls4color = '#7F8C8D'
    cls5color = '#D98880'
    cls6color = '#FF00FF'
    
    pos = {}
    dos = {}
    colors = []
    #pod=[]
    #ld=[]
    lab=[]
    for v in range(34):
        ld=[]
        
        #print(v)
        #print("all", all_logits[i][v])
        dos[v] = all_logits[i][v].numpy()
        #print(pos[v])
        cls = dos[v].argmax() # returns index of highest 
        lab.append(cls)
        
        #a=np.array((all_logs[i][v]))
        #pos[v] = a
        a=np.array(tsne_data1[i][v])
        pos[v] = tsne_data1[i][v]
        #print("cls",cls)
        #print(pos[v])
        if(cls==0):
            colors.append(cls1color)
        elif(cls==1):
            colors.append(cls2color)
        elif(cls==2):
            colors.append(cls3color)
        elif(cls==3):
            colors.append(cls4color)  
        epoc.append(i)
        #print(pos)
    group.append(lab)
        #colors.append(cls1color if cls else cls2color elif ) # according to the index can give color
    ax.cla()
    ax.axis('off')
    ax.set_title('Epoch: %d' % i)
    #pos = nx.kamada_kawai_layout(nx_G)
    
    #nx.draw_networkx(nx_G.to_undirected(), pos, node_color=colors,
           # with_labels=False, node_size=100, ax=ax)
    nx.draw_networkx(nx_G.to_undirected(), pos, node_color=colors,
           with_labels=False, node_size=50, ax=ax)
    
    #print("eeeeee",pos)
    
fig = plt.figure(dpi=150)
fig.clf()
ax = fig.subplots()
 # draw the prediction of the first epoch

draw(200)
plt.show()

The following animation shows how the model correctly predicts the community
after a series of training epochs.



In [ ]:
ani = animation.FuncAnimation(fig, draw, frames=len(all_logits), interval=201)
ani.save('pickup_animation.gif', writer='imagemagick', fps=10)
#plt.show()

In [ ]:
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
import pandas as pd
i = 0
while i<201:
    outp=all_logits[i]
    data_1000=outp
#labels_1000=labels
    print(group[i])
    md=TSNE(n_components=2,random_state=0,perplexity = 2,learning_rate  = 100, n_iter=5000)

    tsne_data1=md.fit_transform(data_1000)#Apply TSNE on the 190 iter
    tsne_data1=np.vstack((labels,tsne_data1.T)).T #Stack 
    tsne_df=pd.DataFrame(data=tsne_data1,columns=["label","Dim_1","Dim_2"])
    tsne_df["Epochs"] = i
    #print(tsne_df)
    #print(tsne_df)
    #with open("output.csv", 'w+') as f:
        #tsne_df.to_csv(f, header=False)
    export_csv = tsne_df.to_csv ('files190.csv',mode="a", index = None, header=False)
    print("iteraction - ", i)
    i = i+10